<a href="https://colab.research.google.com/github/bundickm/miracle-messages-map-DS/blob/master/Michael_Bundick_Miracle_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from re import search
pd.set_option('max_colwidth', 3000)
pd.options.display.max_rows = 5000

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/mkirby1995/miracle_messages/master/Master%20MM%20Volunteers%20-%20MM%20Volunteers.csv')

In [167]:
df[['City','Stateprovince','Country','ZIP']].isnull().sum()/7097

City             0.333521
Stateprovince    0.511484
Country          0.458785
ZIP              0.929266
dtype: float64

In [0]:
def strip_nan():
  for col in ['City','Stateprovince','Country','Full Address']:
    df[col] = df[col].str.strip()
    df[col] = df[col].str.strip('.')
    df.loc[df[col] == '', col] = np.NaN

def try_join(l):
  try:
    return ' '.join(map(str, l))
  except TypeError:
    return np.NaN

In [0]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'}

can_provinces = {'AB':'Alberta',
                 'BC':'British Columbia',
                 'MB':'Manitoba',
                 'NB':'New Brunswick',
                 'NL':'Newfoundland',
                 'NS':'Nova Scotia',
                 'ON':'Ontario',
                 'PE':'Prince Edward Island',
                 'QC':'Quebec',
                 'SK':'Saskatchewan',
                 'NT':'Northwest Territories',
                 'NU':'Nunavut',
                 'YT':'Yukon'}

US_cities = ['Boston', 'Los Angeles', 'San Diego', 'San Francisco', 
             'Salt Lake City', 'Chicago', 'New York City', 'Dallas', 
             'Cincinnati', 'Baltimore', 'Seattle', 'Philadelphia', 
             'New Orleans', 'Nashville', 'Pittsburgh', 'Memphis',
             'Charlottesville', 'Knoxville', 'Miami']

countries = {'United States':'USA',
             'US':'USA',
             'United States of America':'USA',
             'CAN':'Canada',
             'UK':'United Kingdom',
             'AU':'Australia',
             'AUS':'Australia',
             'GB':'United Kingdom'}

In [0]:
def city_state_country_clean():
  strip_nan()
  
  #Pick out States from full address, fill matching empty 'Stateprovince' slot
  #Convert State Abbreviations to Full Name
  for abr in states:
    df.loc[(df['Full Address'].str.contains(states[abr]) & 
            df['Stateprovince'].isnull()) | 
           (df['Stateprovince'].str.lower() == abr.lower()),'Stateprovince'] = states[abr]

  #Same thing but with Canadian Provinces
  for abr in can_provinces:
    df.loc[(df['Full Address'].str.contains(can_provinces[abr]) & 
           df['Stateprovince'].isnull()) | 
           (df['Stateprovince'].str.lower() == abr.lower()),'Stateprovince'] = can_provinces[abr]

  #Find Major US Cities with Unique Names, fill in city and country
  for city in US_cities:
    df.loc[df['Full Address'].str.contains(city) & 
           df['City'].isnull(), 'City'] = city
    df.loc[df['Full Address'].str.contains(city) & 
           df['Country'].isnull(), 'Country'] = 'USA'

  #Get City, and if statprovince is empty, dump everything after first comma there
  temp = df['City'].str.split(',')
  df['City'] = temp.str[0]
  mask = df['Stateprovince'].isnull()
  df.loc[mask, 'Stateprovince'] = temp.loc[mask].str[1:]
  df.loc[mask, 'Stateprovince'] = [try_join(l) for l in 
                                   df.loc[mask, 'Stateprovince']]
  strip_nan()

  #Set Country of US States to 'USA'
  for abr in states:
    df.loc[df['Stateprovince'].str.lower() == states[abr].lower(), 'Country'] = 'USA'

  #Set Country of CAN Provinces to 'Canada'
  for abr in can_provinces:
    df.loc[df['Stateprovince'].str.lower() == can_provinces[abr].lower(), 'Country'] = 'Canada'

  #US Postal Codes from Full Address
  for i in df['Full Address'].index:
    if ((df[df.index == i]['ZIP'].apply(type).any() != str) &
        (df[df.index == i]['Full Address'].apply(type).any() == str)):
      if search('\d{5}', df.loc[i, 'Full Address']):
        df.loc[i, 'ZIP'] = search('\d{5}', df.loc[i, 'Full Address']).group(0)

  #Convert abbreviations to full country name
  for abr in countries:
    df.loc[df['Country'].str.lower() == abr.lower(), 'Country'] = countries[abr]
    df.loc[df['Stateprovince'].str.lower() == abr.lower(), 'Country'] = countries[abr]
  
  strip_nan()

In [0]:
city_state_country_clean()

#City
df.loc[df.index == 730, 'City'] = 'Belgrade'
df.loc[df.index == 2793, 'City'] = 'Bridgeport'
df.loc[df.index == 733, 'City'] = 'Belton'
df.loc[df.index == 912, 'City'] = 'Buffalo'
df.loc[df.index == 929, 'City'] = 'Byron Bay'
df.loc[df.index == 1483, 'City'] = 'Columbus'
df.loc[df.index == 1571, 'City'] = 'Daytona Beach'
df.loc[df.index == 1585, 'City'] = 'Denton'
df.loc[df.index == 1653, 'City'] = 'Detroit'
df.loc[df.index == 1735, 'City'] = 'Fallbrook'
df.loc[df.index == 2039, 'City'] = 'Houston'
df.loc[df.index == 2187, 'City'] = 'Monroe'
df.loc[df.index == 2243, 'City'] = 'Las Vegas'
df.loc[df.index == 2721, 'City'] = 'Montreal'
df.loc[df.index == 2784, 'City'] = 'Nelson'
df.loc[df.index == 2803, 'City'] = 'New Orleans'
df.loc[df.index == 2810, 'City'] = 'Orlando'
df.loc[df.index == 2987, 'City'] = 'Philadelphia'
df.loc[df.index == 3015, 'City'] = 'Fremont'
df.loc[df.index == 3184, 'City'] = 'Orange County'
df.loc[df.index == 3451, 'City'] = 'Raleigh'
df.loc[df.index == 3571, 'City'] = 'Salt Lake City'
df.loc[((df.index == 3613) | (df.index == 4517)), 'City'] = 'San Diego'
df.loc[df.index == 3958, 'City'] = 'Santa Cruz'
df.loc[df.index == 3978, 'City'] = 'Santa Rosa'
df.loc[df.index == 4034, 'City'] = 'Seattle'
df.loc[df.index == 4108, 'City'] = 'Lake Tahoe'
df.loc[df.index == 4182, 'City'] = 'Stockholm'
df.loc[df.index == 4183, 'City'] = 'Stony Brook'
df.loc[df.index == 4186, 'City'] = 'Stuart'
df.loc[df.index == 4243, 'City'] = 'Tampa'
df.loc[df.index == 4334, 'City'] = 'Spokane'
df.loc[df.index == 4413, 'City'] = 'Brighton'
df.loc[df.index == 4497, 'City'] = 'Victoria'
df.loc[((df.index == 4506) | (df.index == 2983)), 'City'] = 'Virginia Beach'
df.loc[df.index == 4548, 'City'] = 'Warrington'
df.loc[((df.index == 4571) | (df.index == 3434)), 'City'] = 'Washington'
df.loc[df.index == 4571, 'City'] = 'Williamsport'
df.loc[df.index == 4668, 'City'] = 'York'
df.loc[df.index == 4678, 'City'] = 'Stanford'
df.loc[df.index == 4716, 'City'] = 'Arlington'
df.loc[df.index == 5896, 'City'] = 'Burlingame'
df.loc[df.index == 6892, 'City'] = 'Oakland'
df.loc[df.index == 212, 'City'] = 'Monrovia'
df.loc[df.index == 278, 'City'] = 'Nashville'
df.loc[df.index == 333, 'City'] = 'Pittsburgh'
df.loc[df.index == 349, 'City'] = 'Memphis'
df.loc[df.index == 355, 'City'] = 'Toledo'
df.loc[df.index == 539, 'City'] = 'Alexandria'
df.loc[df.index == 631, 'City'] = 'Nevada City'
df.loc[df.index == 699, 'City'] = 'Baltimore'
df.loc[df.index == 702, 'City'] = 'Bangor'
df.loc[df.index == 837, 'City'] = 'Boulder City'
df.loc[df.index == 852, 'City'] = 'Brasilia'
df.loc[df.index == 903, 'City'] = 'Bryan'
df.loc[df.index == 909, 'City'] = 'Buenos Aires'
df.loc[df.index == 924, 'City'] = 'Burlington'
df.loc[df.index == 1283, 'City'] = 'Cambridge'
df.loc[df.index == 1293, 'City'] = 'Canton'
df.loc[df.index == 1308, 'City'] = 'Carrollton'
df.loc[df.index == 1328, 'City'] = 'Knoxville'
df.loc[df.index == 1344, 'City'] = 'Charlottesville'
df.loc[df.index == 1348, 'City'] = 'Chelmsford'
df.loc[df.index == 1453, 'City'] = 'Collierville'
df.loc[df.index == 1839, 'City'] = 'Frankfurt'
df.loc[df.index == 1916, 'City'] = 'Aachen'
df.loc[df.index == 4665, 'City'] = 'Novato'
df.loc[df.index == 3531, 'City'] = 'Wallaceburg'
df.loc[df.index == 2523, 'City'] = 'Cork'
df.loc[df.index == 2100, 'City'] = 'Ipswich'
df.loc[df.index == 4612, 'City'] = 'Roby Hill'
df.loc[df.index == 1980, 'City'] = 'New London'
df.loc[df.index == 1995, 'City'] = 'Hesperia'
df.loc[df.index == 1995, 'City'] = 'Kampala'
df.loc[df.index == 2730, 'City'] = 'Moscow'
df.loc[df.index == 1917, 'City'] = 'Saarbrucken'
df.loc[((df.index == 3130) | (df.index == 1430)), 'City'] = 'Cleveland'
df.loc[df.index == 3170, 'City'] = 'Huntington'
df.loc[df.index == 3205, 'City'] = 'Huntington'
df.loc[df.index == 4092, 'City'] = 'Singapore'
df.loc[df.index == 2127, 'City'] = 'Johannesburg'
df.loc[df.index == 319, 'City'] = 'New Mexico'
df.loc[df.index == 2138, 'City'] = 'Kamloomps'
df.loc[df.index == 3982, 'City'] = 'Santa Rosa'
df.loc[df.index == 4999, 'City'] = 'Escondido'
df.loc[df.index == 13, 'City'] = 'Sousse'
df.loc[df.index == 869, 'City'] = np.NaN
df.loc[((df.index == 1469) | (df.index == 1470)), 'City'] = 'Columbia'
df.loc[((df.index == 839) | (df.index == 840)), 'City'] = 'Boulder'
df.loc[((df.index == 1289) | (df.index == 4449)), 'City'] = 'Vancouver'
df.loc[((df.index == 3223) | (df.index == 3437)), 'City'] = 'Ottawa'
df.loc[((df.index == 2631) | (df.index == 2632)), 'City'] = 'Miami'

df.loc[~df['City'].isnull() & 
       df['Full Address'].str.contains('London'), 'City'] = 'London'
df.loc[df['City'].isnull() & 
       df['Full Address'].str.contains('London'), 'City'] = 'London'
df.loc[((df.index == 2333) | (df.index == 708) | (df.index == 4327)), 'City'] = 'London'

Portland = [412]
for index in Portland:
  df.loc[df.index == index, 'City'] = 'Portland'

LA = [2357, 2360, 189, 251]
for index in LA:
  df.loc[df.index == index, 'City'] = 'Los Angeles'

Chicago = [1400, 1403, 2263, 1948, 3106, 4185]
for index in Chicago:
  df.loc[df.index == index, 'City'] = 'Chicago'

Boston = [2957, 827, 1495]
for index in Boston:
  df.loc[df.index == index, 'City'] = 'Boston'

Dallas = [1270, 2055, 72]
for index in Dallas:
  df.loc[df.index == index, 'City'] = 'Dallas'

df.loc[df['City'].isnull() & 
       df['Full Address'].str.contains('Toronto')] = 'Toronto'
Toronto = [4286, 4310, 4315, 4321]
for index in Toronto:
  df.loc[df.index == index, 'City'] = 'Toronto'

SF = [3244, 3673, 3674, 4419, 4420, 7059, 7066, 155]
for index in SF:
  df.loc[df.index == index, 'City'] = 'San Francisco'

NYC = [2883, 895, 2936, 3002, 3089, 891, 896, 1968]
for index in NYC:
  df.loc[df.index == index, 'City'] = 'New York City'

#State
df.loc[df.index == 1917, 'Stateprovince'] = 'Saarland'
df.loc[df.index == 730, 'Stateprovince'] = 'Belgrade'
df.loc[df.index == 4337, 'Stateprovince'] = 'Nova Scotia'
df.loc[df.index == 4609, 'Stateprovince'] = 'Alberta'
df.loc[df.index == 4655, 'Stateprovince'] = 'Manitoba'
df.loc[df.index == 319, 'Stateprovince'] = 'New Mexico'
df.loc[((df.index == 2557) | (df.index == 4111) | (df.index == 5199)), 'Stateprovince'] = 'New Jersey'
df.loc[((df.index == 1920) | (df.index == 4199)), 'Stateprovince'] = 'Queensland'
df.loc[((df.index == 839) | (df.index == 3432)), 'Stateprovince'] = 'Colorado'
df.loc[((df.index == 1653) | (df.index == 2743)), 'Stateprovince'] = 'Michigan'
df.loc[df.index == 2187, 'Stateprovince'] = 'New Hampshire'
df.loc[((df.index == 2721) | (df.index == 2722)), 'Stateprovince'] = 'Quebec'
df.loc[df.index == 3170, 'Stateprovince'] = 'West Virginia'
df.loc[df.index == 4182, 'Stateprovince'] = 'Uppland'
df.loc[df.index == 4413, 'Stateprovince'] = 'Sussex'
df.loc[df.index == 2100, 'Stateprovince'] = 'Suffolk'
df.loc[df.index == 1348, 'Stateprovince'] = 'Essex'
df.loc[df.index == 4571, 'Stateprovince'] = 'District of Columbia'
df.loc[df.index == 702, 'Stateprovince'] = 'Maine'
df.loc[df.index == 852, 'Stateprovince'] = 'Central West'
df.loc[df.index == 909, 'Stateprovince'] = 'Buenos Aires'
df.loc[((df.index == 924) | (df.index == 2999)), 'Stateprovince'] = 'Vermont'
df.loc[df.index == 1802, 'Stateprovince'] = 'Georgia'
df.loc[df.index == 1839, 'Stateprovince'] = 'Hesse'
df.loc[((df.index == 4007) | (df.index == 4187) | (df.index == 4599)), 'Stateprovince'] = 'Wisconsin'
df.loc[df.index == 1860, 'Stateprovince'] = 'South Carolina'
df.loc[df.index == 1910, 'Stateprovince'] = 'Geneva'
df.loc[df.index == 2523, 'Stateprovince'] = 'Munster'
df.loc[df.index == 4092, 'Stateprovince'] = 'Singapore'
df.loc[df.index == 2730, 'Stateprovince'] = 'Central'
df.loc[df.index == 2127, 'Stateprovince'] = 'Guateng'
df.loc[df.index == 3, 'Stateprovince'] = 'Indiana'
df.loc[df.index == 1965, 'Stateprovince'] = 'Hamburg'
df.loc[df.index == 1971, 'Stateprovince'] = 'Lower Saxony'
df.loc[df.index == 138, 'Stateprovince'] = 'tbilisi'
df.loc[df.index == 3447, 'Stateprovince'] = 'North Carolina'
df.loc[df.index == 2166, 'Stateprovince'] = 'Wakiso District'
df.loc[df.index == 1916, 'Stateprovince'] = 'North Rhine-Westphalia'
df.loc[df.index == 2208, 'Stateprovince'] = 'Zurich'
df.loc[df.index == 13, 'Stateprovince'] = 'Sousse'
df.loc[((df.index == 1483) | (df.index == 355) | (df.index == 3130)), 'Stateprovince'] = 'Ohio'
df.loc[((df.index == 4034) | (df.index == 4334)), 'Stateprovince'] = 'Washington'
df.loc[((df.index == 2187) | (df.index == 2803)), 'Stateprovince'] = 'Louisiana'
df.loc[((df.index == 3433) | (df.index == 2746)), 'Stateprovince'] = 'Maharashtra'
df.loc[df['City'] == 'Montreal','Stateprovince'] = 'Quebec'
df.loc[df['City'] == 'Barcelona','Stateprovince'] = 'Barcelona'
df.loc[df['City'] == 'Paris','Stateprovince'] = 'Paris'

Maryland = [699, 4717, 2709, 4417]
for index in Maryland:
  df.loc[df.index == index, 'Stateprovince'] = 'Maryland'
  
Oregon = [4136, 412]
for index in Oregon:
  df.loc[df.index == index, 'Stateprovince'] = 'Oregon'

Connecticut = [1561, 2793, 1922, 1980, 4645]
for index in Connecticut:
  df.loc[df.index == index, 'Stateprovince'] = 'Connecticut'

df.loc[df['City'] == 'Chicago', 'Stateprovince'] = 'Illinois'
Illinois = [1400, 1403, 2263, 1948, 3106, 4185]
for index in Illinois:
  df.loc[df.index == index, 'Stateprovince'] = 'Illinois'

df.loc[(df['City'] == 'London') & 
       (df['Country'] == 'United Kingdom'), 'Stateprovince'] = 'England'
England = [0, 2326, 2327, 2328, 2329, 2330, 2331, 4117, 4118, 4400, 4612]
for index in England:
  df.loc[df.index == index, 'Stateprovince'] = 'England'
  
NSW = [929, 4224, 4225, 4226, 4211]
for index in NSW:
  df.loc[df.index == index, 'Stateprovince'] = 'New South Wales'
  
Massachusetts = [2957, 827, 1495]
for index in Massachusetts:
  df.loc[df.index == index, 'Stateprovince'] = 'Massachusetts'
  
Arkansas = [1826, 1269, 1740]
for index in Arkansas:
  df.loc[df.index == index, 'Stateprovince'] = 'Arkansas'
  
Utah = [3430, 3571, 1529, ]
for index in Utah:
  df.loc[df.index == index, 'Stateprovince'] = 'Utah'
  
Tennessee = [278, 349, 1328, 1344, 1453, 2604, ]
for index in Tennessee:
  df.loc[df.index == index, 'Stateprovince'] = 'Tennessee'
  
Virginia = [4506, 4716, 539, 2983]
for index in Virginia:
  df.loc[df.index == index, 'Stateprovince'] = 'Virginia'
  
Pennsylvania = [2987, 4605, 4668, 333, 3352, 3353]
for index in Pennsylvania:
  df.loc[df.index == index, 'Stateprovince'] = 'Pennsylvania'

df.loc[df['City'] == 'Toronto', 'Stateprovince'] = 'British Columbia'
BC = [1289, 2784, 4449, 4497, 2748, 2749, 3344, 4202, 4453, 4456,
      4457, 4458, 4459, 4460, 4461, 4462, 4463, 4464, 4465, 4466, 4467,
      4468, 4469, 4470, 4471, 4472, 4473, 4477, 4498, 4499, 4500, 2138, 4450]
for index in BC:
  df.loc[df.index == index, 'Stateprovince'] = 'British Columbia'

df.loc[df['Stateprovince'].isnull() & 
       df['Full Address'].str.contains('Ontario')] = 'Ontario'
Ontario = [3223, 3437, 4286, 4310, 4315, 4321, 2652, 1956, 2166, 3219, 3225,
           3224, 3230, 3531, 4269, 4271, 4314, 4316, 4318, 4319, 4320, 4647,
           4322, 4323, 4324, 4325]
for index in Ontario:
  df.loc[df.index == index, 'Stateprovince'] = 'Ontario'
  
Nevada = [2243, 3479, 4108, ]
for index in Nevada:
  df.loc[df.index == index, 'Stateprovince'] = 'Nevada'
  
Florida = [1571, 2599, 2631, 2632, 2810, 4186, 4243, 2771, 3205]
for index in Florida:
  df.loc[df.index == index, 'Stateprovince'] = 'Florida'
  
Texas = [733, 1270, 2039, 2055, 1308, 3272, 4085]
for index in Texas:
  df.loc[df.index == index, 'Stateprovince'] = 'Texas'

df.loc[((df['City'] == 'Los Angeles') | (df['City'] == 'San Francisco') | 
        (df['City'] == 'San Diego')), 'Stateprovince'] = 'California'
California = [2357, 2360, 3184, 3244, 3613, 3673, 3674, 3958, 3978, 4419, 4420,
              4678, 5896, 7059, 7066, 6892, 212, 631, 2726, 2563, 2549, 2198,
              3247, 3253, 3372, 3971, 3988, 3999, 4189, 4326, 4516, 4517, 4659,
              4665, 1995, 155, 189, 251, 3982, 4999]
for index in California:
  df.loc[df.index == index, 'Stateprovince'] = 'California'

NY = [2883, 895, 912, 2936, 3002, 3089, 4183, 1968, 2000,]
for index in NY:
  df.loc[df.index == index, 'Stateprovince'] = 'New York'

#Country
df.loc[df.index == 730, 'Country'] = 'Serbia'
df.loc[df.index == 138, 'Country'] = 'Georgia'
df.loc[df.index == 2127, 'Country'] = 'South Africa'
df.loc[df.index == 1582, 'Country'] = 'Denmark'
df.loc[df.index == 4182, 'Country'] = 'Sweden'
df.loc[((df.index == 3526) | (df.index == 4265) | (df.index == 4266)), 'Country'] = 'Netherlands'
df.loc[df.index == 2730, 'Country'] = 'Russia'
df.loc[df.index == 4258, 'Country'] = 'Iran'
df.loc[df.index == 4260, 'Country'] = 'Israel'
df.loc[df.index == 2166, 'Country'] = 'Uganda'
df.loc[(df.index == 1669) | (df.index == 2523), 'Country'] = 'Ireland'
df.loc[(df.index == 7067), 'Country'] = 'Vietnam'
df.loc[(df.index == 4092), 'Country'] = 'Singapore'
df.loc[(df.index == 2735), 'Country'] = 'New Zealand'
df.loc[(df.index == 909), 'Country'] = 'Argentina'
df.loc[(df.index == 2584), 'Country'] = 'Colombia'
df.loc[(df.index == 13), 'Country'] = 'Tunisia'
df.loc[((df.index == 3433) | (df.index == 2746) | (df.index == 2737)), 'Country'] = 'India'
df.loc[((df.index == 1910) | (df.index == 2208)), 'Country'] = 'Switzerland'

USA = [733, 827, 839, 895, 912, 1400, 1403, 1269, 1270, 1483, 1561, 1571, 1653,
       2187, 2243, 2357, 2360, 2599, 2631, 2632, 2803, 2810, 2883, 2936, 2987,
       3002, 3089, 3184, 3244, 3430, 3479, 3571, 3613, 3673, 3674, 3958, 3978,
       4034, 4108, 4183, 4186, 4243, 4334, 4419, 4420, 4506, 4571, 4605, 4668,
       4678, 4716, 5896, 6892, 7059, 7066, 212, 278, 333, 349, 355, 631, 699,
       702, 924, 1308, 1328, 1344, 1453, 1495, 1529, 1740, 1802, 1860, 2793,
       2771, 2726, 2563, 2549, 2198, 3247, 3253, 3372, 3971, 3988, 3999, 4189,
       4326, 4516, 4517, 4659, 4665, 2743, 4717, 2709, 2604, 2557, 2263, 1948,
       3106, 1922, 1922, 1980, 4645, 1995, 2999, 3130, 3170, 3205, 3272, 3352,
       3353, 3432, 3434, 3447, 4007, 4111, 4185, 4136, 4187, 155, 189, 251, 319,
       412, 1430, 3982, 4417, 4999, 5199]
for index in USA:
  df.loc[df.index == index, 'Country'] = 'USA'

CANADA = [1289, 1694, 1891, 2721, 2784, 3223, 3437, 4152, 4286, 4291, 4298,
          4310, 4315, 4321, 4449, 4497, 2748, 2749, 3344, 4202, 4453, 4456,
          4457, 4458, 4459, 4460, 4461, 4462, 4463, 4464, 4465, 4466, 4467,
          4468, 4469, 4470, 4471, 4472, 4473, 4477, 4498, 4499, 4500, 2652, 
          1956, 2166, 3219, 3225, 3224, 3230, 3531, 4269, 4271, 4314, 4316,
          4318, 4319, 4320, 4647, 4322, 4323, 4324, 4325, 3173, 3174, 3175,
          3221, 3222, 4337, 4609, 4655, 2138, 4450]
for index in CANADA:
  df.loc[df.index == index, 'Country'] = 'Canada'

Australia = [929, 1920, 1925, 3278, 3280, 3378, 4199, 4212, 4213, 4214, 4215,
             4216, 4217, 4218, 4219, 4220, 4221, 4222, 4224, 4225, 4226, 4496,
             4211]
for index in Australia:
  df.loc[df.index == index, 'Country'] = 'Australia'

df.loc[df['Stateprovince'] == 'England', 'Country'] = 'United Kingdom'
UK = [1705, 2332, 2333, 3524, 3532, 4413, 4664, 708, 1348, 2326, 2327, 2328,
      2329, 2330, 2331, 4117, 4118, 4119, 4400, 4612, 2254, 4009, 4327, 4648, 
      2100, ]
for index in UK:
  df.loc[df.index == index, 'Country'] = 'United Kingdom'

Germany = [1839, 1916, 1917, 1965, 1971 ]
for index in Germany:
  df.loc[df.index == index, 'Country'] = 'Germany'
  
France = [2705, 3259, 3262, 3473, 4328, 3260, 3261, ]
for index in France:
  df.loc[df.index == index, 'Country'] = 'France'

Italy = [3521, 3522, 3523]
for index in Italy:
  df.loc[df.index == index, 'Country'] = 'Italy'

Chile = [3991, 3992, 3993]
for index in Chile:
  df.loc[df.index == index, 'Country'] = 'Chile'

#Zip
df.loc[df.index == 3247, 'ZIP'] = '90272'
df.loc[df.index == 3372, 'ZIP'] = '91767'
df.loc[df.index == 4189, 'ZIP'] = '94585'
df.loc[df.index == 4516, 'ZIP'] = '92801'
df.loc[df.index == 2743, 'ZIP'] = '49442'
df.loc[df.index == 4612, 'ZIP'] = 'WN80QP'
df.loc[df.index == 4085, 'ZIP'] = '75090'
df.loc[df.index == 4111, 'ZIP'] = '08882'
df.loc[df.index == 4187, 'ZIP'] = '54235'
df.loc[df.index == 4599, 'ZIP'] = '54981'
df.loc[df.index == 4999, 'ZIP'] = '92025'

city_state_country_clean()

In [0]:
#Fill in Full Address as much as possible using city, state, country
mask = (df['Full Address'].isnull() & 
      ~df['City'].isnull() & 
      ~df['Stateprovince'].isnull() & 
      ~df['Country'].isnull())
df.loc[mask,'Full Address'] = df['City'] + ', ' + df['Stateprovince'] + ', ' + df['Country']

In [0]:
for abr in states:
  df.loc[df['Stateprovince'].isnull() & df['Full Address'].str.contains(abr) & df['Full Address'].str.contains(states[abr]), 'Stateprovince'] = states[abr]

In [175]:
df[['City','Stateprovince','Country','ZIP','Full Address']].isnull().sum()/7097

City             0.314076
Stateprovince    0.351980
Country          0.326617
ZIP              0.916866
Full Address     0.178103
dtype: float64

In [165]:
for abr in states:
  print(df.loc[(df['Stateprovince'] == states[abr]) & (df['Country'] != 'USA'), 'City'])

Series([], Name: City, dtype: object)
523            NaN
545     ALPHARETTA
2692           NaN
Name: City, dtype: object
460    NaN
Name: City, dtype: object
585    NaN
Name: City, dtype: object
675                            NaN
676                            NaN
677                            NaN
678                            NaN
679                            NaN
680                            NaN
3328                           NaN
3329                           NaN
4341    Tucson AZ / Westchester NY
Name: City, dtype: object
420                    NaN
732                    NaN
865                Bristol
914                    NaN
931                    NaN
932                    NaN
933                    NaN
934                    NaN
935                    NaN
936                    NaN
937                    NaN
938                    NaN
939                    NaN
940                    NaN
941                    NaN
942                    NaN
943                    NaN
944  

In [162]:
df[['City','Stateprovince','Country','Full Address']]

,City,Stateprovince,Country,Full Address
0,Coventry,England,United Kingdom,56
1,San Francisco,California,USA,297
2,San Francisco,California,USA,2182
3,NaN,Indiana,USA,46037
4,San Francisco,California,USA,415536472
5,Santa Monica,California,USA,#1 Santa Monica California 90404 United States Santa Monica California USA
6,Cos Cob,Connecticut,USA,#12 Cos Cob CT 6807 US Cos Cob CT
7,Bellevue,Washington,USA,#143 Bellevue WA 98007 United States Bellevue WA USA
8,San Francisco,California,USA,#3 San Francisco California 94122 United States San Francisco TX USA
9,Port Moody,British Columbia,Canada,#410-260 Newport Dr


In [157]:
df[~df['City'].isnull() & df['Stateprovince'].isnull() & ~df['Country'].isnull()][['City','Stateprovince','Country','Full Address']]

,City,Stateprovince,Country,Full Address
39,Canberra,NaN,Australia,"11 lodder place, Chisolm"
68,Saint Genies de Malgoires,NaN,FR,13 avenue des JonquiÃ¨res
98,Cornwall,NaN,United Kingdom,"14a, summerlease crescent, bude"
137,Croydon,NaN,United Kingdom,17 Boulogne Road
170,Hove,NaN,United Kingdom,2 park gate
173,Bensville,NaN,Australia,2 Yowie CL
327,Richmond,NaN,Australia,4/22 Abinger Street
357,Bristol,NaN,United Kingdom,47 Elmore Road
370,Cambridge,NaN,United Kingdom,5 Linden Close
371,Crestmead,NaN,Australia,5 Paddies crescent


In [0]:
df[((df['City'].isnull() | df['Stateprovince'].isnull() | df['Country'].isnull()) & (~df['Full Address'].isnull()))][['City','Stateprovince','Country','Full Address']].shape

(1706, 4)

In [0]:
df[df.index == 4297]['Stateprovince'] == ''

4297    False
Name: Stateprovince, dtype: bool